In [46]:
# Import required packages
import pandas as pd
import json
import numpy as np


In [48]:
# Read the GEOID
with open(r'C:\Users\lukej\OneDrive\Desktop\nyc-busyness\etl\census tract geofiles\manhattan_census_tracts.geojson', 'r') as f:
    geojson = json.load(f)
    
geoids = [feature['properties']['GEOID'] for feature in geojson['features']]
print(f"Found {len(real_geoids)} census tracts")

Found 310 census tracts


In [51]:
# Process historical foot traffic data
hist = pd.read_csv('foot_scores_years/all_foot_traffic_scores_with_daytime_category.csv')
hist['year'] = pd.to_datetime(hist['trip_date']).dt.year

print("Years in historical data before filtering:", sorted(hist['year'].unique()))
hist = hist[(hist['year'] >= 2019) & (hist['year'] <= 2030)]  # Keep only 2019-2023
print("Years in historical data after filtering:", sorted(hist['year'].unique()))

hist['col'] = hist['daytime_category'] + '_' + hist['year'].astype(str)
hist_wide = hist.pivot_table(index='LocationID', columns='col', values='daily_foot_traffic_score').reset_index()

print(f"Historical data shape: {hist_wide.shape}")
print("Historical columns:", [col for col in hist_wide.columns if col != 'LocationID'][:5])

Years in historical data before filtering: [2001, 2002, 2003, 2004, 2008, 2009, 2011, 2019, 2020, 2021, 2022, 2023, 2098]
Years in historical data after filtering: [2019, 2020, 2021, 2022, 2023]
Historical data shape: (67, 16)
Historical columns: ['afternoon_2019', 'afternoon_2020', 'afternoon_2021', 'afternoon_2022', 'afternoon_2023']


In [52]:
# Read future foot traffic data
future = pd.read_csv('future_foot_traffic.csv')
print("Years in future data before filtering:", sorted(future['year'].unique()))
future = future[future['year'] <= 2030]
print("Years in future data after filtering:", sorted(future['year'].unique()))

Years in future data before filtering: [2025, 2026, 2027]
Years in future data after filtering: [2025, 2026, 2027]


In [54]:
# Process future foot traffic data
future['col'] = future['daytime_category'] + '_pred_' + future['year'].astype(str)
future_wide = future.pivot_table(index='LocationID', columns='col', values='predicted_foot_traffic_score').reset_index()

print(f"Future data shape: {future_wide.shape}")
print("Future columns:", [col for col in future_wide.columns if col != 'LocationID'][:5])

Future data shape: (67, 10)
Future columns: ['afternoon_pred_2025', 'afternoon_pred_2026', 'afternoon_pred_2027', 'evening_pred_2025', 'evening_pred_2026']


In [55]:
# Combine historical and future data
foot_traffic = hist_wide.merge(future_wide, on='LocationID', how='outer').fillna(0)
unique_locations = sorted(foot_traffic['LocationID'].unique())
print(f"Combined data shape: {foot_traffic.shape}")
print(f"Unique LocationIDs: {unique_locations}")
print("All columns:", foot_traffic.columns.tolist())

Combined data shape: (67, 25)
Unique LocationIDs: [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]
All columns: ['LocationID', 'afternoon_2019', 'afternoon_2020', 'afternoon_2021', 'afternoon_2022', 'afternoon_2023', 'evening_2019', 'evening_2020', 'evening_2021', 'evening_2022', 'evening_2023', 'morning_2019', 'morning_2020', 'morning_2021', 'morning_2022', 'morning_2023', 'afternoon_pred_2025', 'afternoon_pred_2026', 'afternoon_pred_2027', 'evening_pred_2025', 'evening_pred_2026', 'evening_pred_2027', 'morning_pred_2025', 'morning_pred_2026', 'morning_pred_2027']


In [57]:
# Map LocationIDs to GEOIDs
result_rows = []
locations = sorted(foot_traffic['LocationID'].unique())
tracts_per_location = len(real_geoids) // len(locations)

print(f"Mapping {len(locations)} LocationIDs to {len(real_geoids)} GEOIDs")
print(f"Approximately {tracts_per_location} census tracts per LocationID")

geoid_index = 0
for i, location_id in enumerate(locations):
    location_data = foot_traffic[foot_traffic['LocationID'] == location_id].iloc[0]
    num_geoids = tracts_per_location + (1 if i < len(real_geoids) % len(locations) else 0)
    
    for j in range(num_geoids):
        if geoid_index < len(real_geoids):
            row = {'GEOID': real_geoids[geoid_index]}
            for col in foot_traffic.columns:
                if col != 'LocationID':
                    row[col] = location_data[col]
            result_rows.append(row)
            geoid_index += 1
            
print(f"Created {len(result_rows)} rows for GEOID mapping")

Mapping 67 LocationIDs to 310 GEOIDs
Approximately 4 census tracts per LocationID
Created 310 rows for GEOID mapping


In [58]:
# Create the final dataframe
result_df = pd.DataFrame(result_rows)

In [59]:
# Sort columns
cols = ['GEOID'] + sorted([col for col in result_df.columns if col != 'GEOID'])
result_df = result_df[cols]

In [60]:
# Round values and sort by GEOID
result_df = result_df.round(2).sort_values('GEOID').reset_index(drop=True)

In [62]:
# Display data before normalisation
print("Current data range before normalisation:")
numeric_cols = [col for col in result_df.columns if col != 'GEOID']
for col in numeric_cols[:3]:  # Sample first 3 columns
    print(f"{col}: {result_df[col].min():.3f} - {result_df[col].max():.3f}")

Current data range before normalisation:
afternoon_2019: 1.000 - 1.000
afternoon_2020: 1.000 - 1.620
afternoon_2021: 1.000 - 3.820


In [64]:
# Function to normalise to 1-10 scale
def normalise_to_1_10(series):
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:  # Handle case where all values are the same
        return pd.Series([5.5] * len(series), index=series.index)
    
    # Scale to 0-1, then to 1-10
    normalised = (series - min_val) / (max_val - min_val)
    return normalised * 9 + 1  # Scale to 1-10 range

In [65]:
# Separate columns by time period
morning_cols = [col for col in result_df.columns if 'morning' in col]
afternoon_cols = [col for col in result_df.columns if 'afternoon' in col]
evening_cols = [col for col in result_df.columns if 'evening' in col]

print(f"\nNormalizing {len(morning_cols)} morning, {len(afternoon_cols)} afternoon, {len(evening_cols)} evening columns")


Normalizing 8 morning, 8 afternoon, 8 evening columns


In [73]:
# Create normalised dataframe
normalised_df = result_df[['GEOID']].copy()

In [75]:
# Normalise each time period separately
for col in morning_cols:
    normalised_df[col] = normalise_to_1_10(result_df[col])
    
for col in afternoon_cols:
    normalised_df[col] = normalise_to_1_10(result_df[col])

for col in evening_cols:
    normalised_df[col] = normalise_to_1_10(result_df[col])

In [76]:
# Add average columns for each year
years = []
for col in morning_cols:
    if 'pred_' in col:
        year = col.split('pred_')[1]
        years.append(f'pred_{year}')
    else:
        year = col.split('_')[1]
        years.append(year)

unique_years = sorted(list(set(years)))
print(f"\nAdding average columns for years: {unique_years}")

for year in unique_years:
    morning_col = f'morning_{year}'
    afternoon_col = f'afternoon_{year}'
    evening_col = f'evening_{year}'
    avg_col = f'average_{year}'
    
    if all(col in normalised_df.columns for col in [morning_col, afternoon_col, evening_col]):
        normalised_df[avg_col] = (
            normalised_df[morning_col] + 
            normalised_df[afternoon_col] + 
            normalised_df[evening_col]
        ) / 3
        print(f"Added {avg_col}")


Adding average columns for years: ['2019', '2020', '2021', '2022', '2023', 'pred_2025', 'pred_2026', 'pred_2027']
Added average_2019
Added average_2020
Added average_2021
Added average_2022
Added average_2023
Added average_pred_2025
Added average_pred_2026
Added average_pred_2027


In [77]:
# Round and organize
numeric_cols = [col for col in normalised_df.columns if col != 'GEOID']
normalised_df[numeric_cols] = normalised_df[numeric_cols].round(2)

In [78]:
# Replace result_df with normalized version
result_df = normalised_df

print(f"New data ranges (1-10 scale):")
sample_cols = [col for col in result_df.columns if col != 'GEOID'][:6]
for col in sample_cols:
    print(f"{col}: {result_df[col].min():.2f} - {result_df[col].max():.2f}")

New data ranges (1-10 scale):
morning_2019: 5.50 - 5.50
morning_2020: 1.00 - 10.00
morning_2021: 1.00 - 10.00
morning_2022: 1.00 - 10.00
morning_2023: 5.50 - 5.50
morning_pred_2025: 1.00 - 10.00


In [79]:
print(f"\nFinal columns: {result_df.columns.tolist()}")


Final columns: ['GEOID', 'morning_2019', 'morning_2020', 'morning_2021', 'morning_2022', 'morning_2023', 'morning_pred_2025', 'morning_pred_2026', 'morning_pred_2027', 'afternoon_2019', 'afternoon_2020', 'afternoon_2021', 'afternoon_2022', 'afternoon_2023', 'afternoon_pred_2025', 'afternoon_pred_2026', 'afternoon_pred_2027', 'evening_2019', 'evening_2020', 'evening_2021', 'evening_2022', 'evening_2023', 'evening_pred_2025', 'evening_pred_2026', 'evening_pred_2027', 'average_2019', 'average_2020', 'average_2021', 'average_2022', 'average_2023', 'average_pred_2025', 'average_pred_2026', 'average_pred_2027']


In [81]:
# Save to CSV
result_df.to_csv('brickwyze_foot_traffic-real_geoid.csv', index=False)